In [63]:
import pandas as pd

artigo_treino = pd.read_csv("https://raw.githubusercontent.com/Yuriferr/Treinamento-Processamento-de-Linguagem-Natural/refs/heads/main/dados/treino.csv")
artigo_teste = pd.read_csv("https://raw.githubusercontent.com/Yuriferr/Treinamento-Processamento-de-Linguagem-Natural/refs/heads/main/dados/teste.csv")

In [64]:
artigo_treino.head()

,title,text,date,category,subcategory,link
0,Opinião do Especialista: Os Desafios e Oportun...,"Lorem ipsum dolor sit amet, consectetur adipis...",2023-01-01,Tecnologia,Hardware,http://noticias.com/artigo/tecnologia/hardware/1
1,Inovação e Perspectivas: O Papel de Pandemia n...,A recente pesquisa sobre Pandemia revelou dado...,2023-01-02,Saúde,Pandemia,http://noticias.com/artigo/saúde/pandemia/2
2,Opinião do Especialista: Os Desafios e Oportun...,O debate sobre Comércio Exterior tem gerado in...,2023-01-03,Economia,Comércio Exterior,http://noticias.com/artigo/economia/comércio-e...
3,Debate Aberto: Vacinação entre o Progresso e o...,"No cenário atual, a questão de Vacinação tem g...",2023-01-04,Saúde,Vacinação,http://noticias.com/artigo/saúde/vacinação/4
4,Debate Aberto: Reforma Educacional entre o Pro...,"No cenário atual, a questão de Reforma Educaci...",2023-01-05,Educação,Reforma Educacional,http://noticias.com/artigo/educação/reforma-ed...


In [65]:
artigo_teste.head()

,title,text,date,category,subcategory,link
0,Crise ou Oportunidade? A Dinâmica de Relações ...,"Lorem ipsum dolor sit amet, consectetur adipis...",2023-01-01,Política,Relações Internacionais,http://noticias.com/artigo/política/relações-i...
1,Opinião do Especialista: Os Desafios e Oportun...,A recente pesquisa sobre Bem-estar revelou dad...,2023-01-02,Saúde,Bem-estar,http://noticias.com/artigo/saúde/bem-estar/2
2,Análise aprofundada: O Impacto de Emprego no C...,O debate sobre Emprego tem gerado intensas dis...,2023-01-03,Economia,Emprego,http://noticias.com/artigo/economia/emprego/3
3,Debate Aberto: Desenvolvimento de Software ent...,"No cenário atual, a questão de Desenvolvimento...",2023-01-04,Tecnologia,Desenvolvimento de Software,http://noticias.com/artigo/tecnologia/desenvol...
4,Crise ou Oportunidade? A Dinâmica de Emprego e...,O debate sobre Emprego tem gerado intensas dis...,2023-01-05,Economia,Emprego,http://noticias.com/artigo/economia/emprego/5


In [66]:
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 23.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [67]:
import spacy
nlp = spacy.load("pt_core_news_sm")

In [68]:
textos_para_tratamento = (titulos.lower() for titulos in artigo_treino.title)

In [69]:
def trata_textos(doc):
  tokens_validos = []


  for token in doc:
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      tokens_validos.append(token.text)


  if len(tokens_validos) > 2:
    return " ".join(tokens_validos)

In [70]:
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                        batch_size = 1000,
                                                        n_process = -1)]

In [71]:
titulos_tratados = pd.DataFrame({"titulo": textos_tratados})

titulos_tratados.head()

,titulo
0,opinião especialista desafios oportunidades ha...
1,inovação perspectivas papel pandemia transform...
2,opinião especialista desafios oportunidades co...
3,debate aberto vacinação progresso desafios éticos
4,debate aberto reforma educacional progresso de...


In [72]:
!pip install gensim

# Spacy

In [73]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

In [74]:
print(len(titulos_tratados))

titulos_tratados = titulos_tratados.dropna().drop_duplicates()

print(len(titulos_tratados))

2000
330


In [75]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [76]:
w2v_modelo.build_vocab(lista_lista_tokens)

In [77]:
w2v_modelo.corpus_count

330

In [78]:
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec

# iniciando a chamada callback
class callback(CallbackAny2Vec):
  def __init__(self):
    self.epoch = 0

  def on_epoch_end(self, model):
    loss = model.get_latest_training_loss()
    if self.epoch == 0:
        print('Loss após a época {}: {}'.format(self.epoch, loss))
    else:
        print('Loss após a época {}: {}'.format(self.epoch, loss- self.loss_previous_step))
    self.epoch += 1
    self.loss_previous_step = loss

In [79]:
w2v_modelo.train(lista_lista_tokens,
                 total_examples = w2v_modelo.corpus_count,
                 epochs = 30,
                 compute_loss = True,
                 callbacks=[callback()])

Loss após a época 0: 3566.992919921875
Loss após a época 1: 3695.214111328125
Loss após a época 2: 3643.3720703125
Loss após a época 3: 3620.29296875
Loss após a época 4: 3707.8642578125
Loss após a época 5: 3600.814453125
Loss após a época 6: 3715.27734375
Loss após a época 7: 3479.607421875
Loss após a época 8: 3415.220703125
Loss após a época 9: 3017.93359375
Loss após a época 10: 2748.6875
Loss após a época 11: 2588.40234375
Loss após a época 12: 2586.21484375
Loss após a época 13: 2522.59375
Loss após a época 14: 2515.80078125
Loss após a época 15: 2432.58203125
Loss após a época 16: 2469.32421875
Loss após a época 17: 2488.0390625
Loss após a época 18: 2406.734375
Loss após a época 19: 2424.36328125
Loss após a época 20: 2403.8046875
Loss após a época 21: 2411.9375
Loss após a época 22: 2453.04296875
Loss após a época 23: 2473.6796875
Loss após a época 24: 2530.6796875
Loss após a época 25: 2419.1875
Loss após a época 26: 2404.65625
Loss após a época 27: 2397.46875
Loss após a ép

(26058, 67260)

In [80]:
w2v_modelo.wv.most_similar("internacionais")

[('legislação', 0.9981498718261719),
 ('basquete', 0.998140811920166),
 ('futebol', 0.9980713129043579),
 ('olimpíadas', 0.9980576038360596),
 ('sustentabilidade', 0.9980564713478088),
 ('inflação', 0.9980545043945312),
 ('literatura', 0.9980372190475464),
 ('emprego', 0.9979997277259827),
 ('cibersegurança', 0.9979788661003113),
 ('vacinação', 0.997956395149231)]

# SkipGram

In [81]:
w2v_modelo_sg = Word2Vec(sg = 1,
                      window = 5,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo_sg.build_vocab(lista_lista_tokens)

w2v_modelo_sg.train(lista_lista_tokens,
                 total_examples = w2v_modelo_sg.corpus_count,
                 epochs = 30,
                 compute_loss = True,
                 callbacks=[callback()])

Loss após a época 0: 6926.4072265625
Loss após a época 1: 7319.78125
Loss após a época 2: 7514.5009765625
Loss após a época 3: 7154.919921875
Loss após a época 4: 6589.44140625
Loss após a época 5: 5399.19921875
Loss após a época 6: 4764.6171875
Loss após a época 7: 4321.53125
Loss após a época 8: 4634.015625
Loss após a época 9: 5007.90234375
Loss após a época 10: 4698.50390625
Loss após a época 11: 4688.078125
Loss após a época 12: 5557.9375
Loss após a época 13: 4660.2578125
Loss após a época 14: 4222.96875
Loss após a época 15: 5004.3046875
Loss após a época 16: 4382.3671875
Loss após a época 17: 4294.046875
Loss após a época 18: 4434.25
Loss após a época 19: 4374.859375
Loss após a época 20: 4731.765625
Loss após a época 21: 4652.5234375
Loss após a época 22: 4128.7890625
Loss após a época 23: 4666.3515625
Loss após a época 24: 3890.1953125
Loss após a época 25: 3733.21875
Loss após a época 26: 3962.75
Loss após a época 27: 3943.4375
Loss após a época 28: 3988.125
Loss após a époc

(26084, 67260)

In [82]:
w2v_modelo_sg.wv.most_similar("internacionais")

[('sustentabilidade', 0.9883832931518555),
 ('pandemia', 0.9879934787750244),
 ('automobilismo', 0.9872362613677979),
 ('olimpíadas', 0.9870615601539612),
 ('eleições', 0.9867727160453796),
 ('futebol', 0.9861956238746643),
 ('legislação', 0.9860640168190002),
 ('inflação', 0.9856796264648438),
 ('ensino', 0.9855701327323914),
 ('inteligência', 0.9852706789970398)]

# Exportando Modelos

In [83]:
# /content/drive/MyDrive/Colab Notebooks
w2v_modelo.wv.save_word2vec_format("/content/drive/MyDrive/Colab Notebooks/Word2Vec/modelo_cbow.txt", binary=False)
w2v_modelo_sg.wv.save_word2vec_format("/content/drive/MyDrive/Colab Notebooks/Word2Vec/modelo_sg.txt", binary=False)

# Classificação com meu modelo

In [84]:
import spacy
nlp = spacy.load("pt_core_news_sm", disable=["parser", "ner", "tagger", "textcat"])

def tokenizador(texto):
  tokens_validos = []
  doc = nlp(texto)

  for token in doc:
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      tokens_validos.append(token.text.lower())

  return tokens_validos

In [85]:
import numpy as np

def combinacao_de_vetores_por_soma(palavras, modelo):
  vetor_resultante = np.zeros(300)
  for pn in palavras:
    try:
      vetor_resultante =+ modelo.wv.get_vector(pn)
    except KeyError:
      pass

  return vetor_resultante

In [86]:
def matriz_vetores(textos, modelo):
  x = len(textos)
  y = 300
  matriz = np.zeros((x,y))

  for i in range(x):
    palavras = tokenizador(textos.iloc[i])
    matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

  return matriz

In [87]:
matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title, w2v_modelo)
matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title, w2v_modelo)
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)

matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_sg)
print(matriz_vetores_treino_sg.shape)
print(matriz_vetores_teste_sg.shape)

(2000, 300)
(2000, 300)
(2000, 300)
(2000, 300)


In [88]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(modelo, x_treino, y_treino, x_teste, y_teste):
  RL = LogisticRegression(max_iter = 800)
  RL.fit(x_treino, y_treino)
  categorias = RL.predict(x_teste)
  resultados = classification_report(y_teste, categorias)
  print(resultados)
  return RL

In [89]:
RL_cbow = classificador(w2v_modelo,
                        matriz_vetores_treino_cbow,
                        artigo_treino.category,
                        matriz_vetores_teste_cbow,
                        artigo_teste.category)

               precision    recall  f1-score   support

      Cultura       0.00      0.00      0.00       239
     Economia       0.11      0.37      0.17       245
     Educação       0.00      0.00      0.00       230
     Esportes       0.13      0.10      0.11       252
Meio Ambiente       0.00      0.00      0.00       244
     Política       0.16      0.12      0.14       276
        Saúde       0.09      0.14      0.11       252
   Tecnologia       0.13      0.20      0.16       262

     accuracy                           0.12      2000
    macro avg       0.08      0.12      0.09      2000
 weighted avg       0.08      0.12      0.09      2000



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [90]:
RL_cbow = classificador(w2v_modelo_sg,
                        matriz_vetores_treino_sg,
                        artigo_treino.category,
                        matriz_vetores_teste_sg,
                        artigo_teste.category)

               precision    recall  f1-score   support

      Cultura       0.11      0.10      0.10       239
     Economia       0.00      0.00      0.00       245
     Educação       0.10      0.09      0.09       230
     Esportes       0.13      0.10      0.11       252
Meio Ambiente       0.15      0.11      0.13       244
     Política       0.16      0.12      0.14       276
        Saúde       0.12      0.10      0.11       252
   Tecnologia       0.14      0.43      0.21       262

     accuracy                           0.13      2000
    macro avg       0.11      0.13      0.11      2000
 weighted avg       0.12      0.13      0.11      2000



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
